In [1]:
import numpy as np
import librosa
from datasketch import MinHashLSHForest, MinHash

def preprocess_audio(audio_file):
    # Load the audio file
    audio, _ = librosa.load(audio_file)

    # Remove leading and trailing silences
    audio_trimmed, _ = librosa.effects.trim(audio)

    return audio_trimmed

def resize_audio(audio, length):
    if len(audio) > length:
        # Truncate audio if longer than the specified length
        resized_audio = audio[:length]
    else:
        # Pad audio with zeros if shorter than the specified length
        resized_audio = np.pad(audio, (0, length - len(audio)), mode='constant')

    return resized_audio

def generate_audio_fingerprint(audio):
    # Generate fingerprints using MinHash
    fingerprints = []
    mhash = MinHash(num_perm=128)  # Set the desired number of permutations
    for value in audio:
        mhash.update(str(value).encode('utf-8'))
    fingerprints.append(mhash)
    
    return fingerprints

def compare_audio_fingerprints(fingerprints1, fingerprints2):
    # Create LSH index
    lsh = MinHashLSHForest(num_perm=128)  # Set the desired number of permutations

    # Index fingerprints from the first audio
    for i, fp in enumerate(fingerprints1):
        lsh.add(i, fp)

    # Index the forest
    lsh.index()

    # Check for matches in the second audio
    for i, fp in enumerate(fingerprints2):
        matches = lsh.query(fp, k=1)  # Set the value of k (number of nearest neighbors) to 1
        if matches:
            return True

    return False


In [ ]:
import os

# Step 1: Preprocess the audio
audio_file1 = './test_Song.wav'
preprocessed_audio1 = preprocess_audio(audio_file1)
max_length = len(preprocessed_audio1)

# Step 2: Generate audio fingerprint for audio_file1
fingerprints1 = generate_audio_fingerprint(preprocessed_audio1)
print(fingerprints1)

# Step 3: Iterate over the folder of songs
folder_path = './test_songs'
for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        # Preprocess the audio
        audio_path = os.path.join(folder_path, filename)
        preprocessed_audio2 = preprocess_audio(audio_path)

        # Resize the audio
        max_length = max(len(preprocessed_audio1), len(preprocessed_audio2))
        resized_audio = resize_audio(preprocessed_audio2, max_length)

        # Generate audio fingerprint for the current song
        fingerprints2 = generate_audio_fingerprint(resized_audio)

        # Compare audio fingerprints
        if compare_audio_fingerprints(fingerprints1, fingerprints2):
            print(f"The song '{filename}' matches audiofile1:{audio_file1}.")